In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [2]:
PINECONE_API_KEY = "pcsk_5pmLVN_7bjAWb2V6xiwm1ixq9hnBDi5Rfp6Z4WUmSUeiHRUEJCpipZ7pMkNPyJ46MFhxgP"
PINECONE_API_ENV = "us-east-1"

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [4]:
extracted_data = load_pdf("/media/dhruv/Local Disk/Medical_chatbot/DATA/")

In [5]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [6]:
text_chunk = text_split(extracted_data)
print("length of my chunk:", len(text_chunk))

length of my chunk: 5860


In [7]:
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embedding()

/home/dhruv/anaconda3/envs/mchatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [12]:
query_result

[-0.034477315843105316,
 0.031023172661662102,
 0.006734910886734724,
 0.02610892429947853,
 -0.03936195746064186,
 -0.1603025197982788,
 0.06692396104335785,
 -0.006441440898925066,
 -0.04745054617524147,
 0.014758836477994919,
 0.07087532430887222,
 0.055527545511722565,
 0.01919332519173622,
 -0.026251299306750298,
 -0.01010951679199934,
 -0.026940451934933662,
 0.022307397797703743,
 -0.022226639091968536,
 -0.1496926248073578,
 -0.01749303936958313,
 0.007676327601075172,
 0.054352276027202606,
 0.0032544792629778385,
 0.03172592446208,
 -0.08462144434452057,
 -0.029405953362584114,
 0.05159562826156616,
 0.048124104738235474,
 -0.003314818488433957,
 -0.05827919766306877,
 0.04196928068995476,
 0.02221069671213627,
 0.12818878889083862,
 -0.02233896404504776,
 -0.011656257323920727,
 0.06292840093374252,
 -0.03287629410624504,
 -0.09122602641582489,
 -0.031175386160612106,
 0.05269954726099968,
 0.047034841030836105,
 -0.08420310169458389,
 -0.030056146904826164,
 -0.020744822919

In [13]:
import chromadb
from chromadb.utils import embedding_functions
from langchain.embeddings import HuggingFaceEmbeddings

In [15]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from chromadb import Client 
from chromadb.config import Settings

def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embedding()

persist_directory = "new_chroma_db"
collection_name = "medical_chatbot"

client = Client()  

settings = Settings(
    chroma_db_impl="duckdb+parquet",  
    persist_directory=persist_directory
)
chroma_db = Chroma(
    collection_name=collection_name,
    embedding_function=embeddings,
    client=client  
)

texts = [t.page_content for t in text_chunk]  

chroma_db.add_texts(
    texts=texts,
    ids=[str(i) for i in range(len(texts))]
)

print("Embeddings successfully added to the new ChromaDB directory.")


Embeddings successfully added to the new ChromaDB directory.


In [18]:
query = "What are Allergies?"
query_embeddings = embeddings.embed_query(query)

results = chroma_db.similarity_search(query, k=3)

print("Result:", results)


Result: [Document(metadata={}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shortness of breath. Symp-\ntoms of food allergies depend on the tissues most sensi-\ntive to the allergen and whether the allergen spread sys-'), Document(metadata={}, page_content='reactions is triggered by harmless, everyday substances.\nThis is the condition known as allergy, and the offend-\ning substance is called an allergen. Common inhaled\nallergens include pollen, dust, and insect parts from tiny\nhouse mites. Common food allergens include nuts, fish,\nand milk.\nAllergic reactions involve a special set of cells in\nthe immune system known as mast cells. Mast cells\nserve as guards in the tissues where the body meets

In [19]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [20]:

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [22]:
from langchain.llms import CTransformers
llm=CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

Fetching 1 files: 100%|██████████| 1/1 [02:14<00:00, 134.24s/it]


In [23]:
from langchain.chains import RetrievalQA

retriever = chroma_db.as_retriever(search_kwargs={"k": 2})
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs
)

In [24]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

/tmp/ipykernel_23298/4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  Acne vulgaris, commonly known as acne, is a skin disorder that occurs when the sebaceous glands in the face become inflamed.
Response :  You should consult a doctor if you have persistent symptoms of furunculosis, such as ringing or buzzing in your ears. This could be a sign of an underlying disease, such as diabetes, and a medical professional can help you determine the cause and appropriate treatment.


KeyboardInterrupt: 

In [ ]:
from pinecone.grpc import PineconGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_2mEEER_26E5t9SNGU1v5H4J4DCqePMRZLp4rkhvyvMfpFUEjvoqvqsAu93qWhqEb8cMjnS")

index_name = "medical_chatbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)